In [2]:
pip install pytaxize

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Chargement des librairies

from pytaxize import Ids
import warnings
import sys
from pytaxize.itis import hierarchy_full
from pytaxize import scicomm
import pandas as pd
# Ca ne marche pas
# from pytaxize.ncbi import hierarchy
# from pytaxize import Classification

In [5]:
def hierarchy(ids):
    """
    Get a full taxonomic hierarchy from NCBI

    :param ids: one or more NCBI taxonomy ids

    :note: Remember to set your Entrez API key as `ENTREZ_KEY`

    :return: dict, named with ids given to `ids`, 
        where each value in the dict is a list of taxa, each
        a dict with the fields ``ScientificName``, ``Rank``, and ``TaxId``

    Usage::

        from pytaxize import ncbi
        ncbi.hierarchy(ids=9606)
        ncbi.hierarchy(ids=[9606,55062,4231])
    """
    toget = ["ScientificName", "Rank", "TaxId"]
    key = os.environ.get("ENTREZ_KEY")
    if key is None:
        raise Exception("ENTREZ_KEY is not defined")
    if not isinstance(ids, list):
        ids = [ids]
    idz = ",".join([str(x) for x in ids])
    args = {"db": "taxonomy", "ID": idz, "api_key": key}
    res = _entrez("efetch", args)
    taxa = res.xpath("//TaxaSet/Taxon")
    out = []
    for x in range(len(taxa)):
        nodes = taxa[x].xpath(".//LineageEx/Taxon")
        tmp = [
            dict(zip(toget, [node.xpath(w)[0].text for w in toget])) for node in nodes
        ]
        tmp.append(dict(zip(toget, [taxa[x].xpath(w)[0].text for w in toget])))
        out.append(tmp)
    return dict(zip(ids, out))

class Classification(object):
    """
    Classification: Retrieve taxonomic hierarchy for taxonomic IDs

    Usage::

        from pytaxize import Classification
        
        # ITIS
        ## one id
        x = Classification(99208)
        x
        x.ids
        res = x.itis()
        res[99208]
        
        ## many ids - with one invalid id
        x = Classification([99208, 129313, 9999999])
        x
        x.ids
        res = x.itis()
        res[99208]
        res[129313]
        res[9999999]

        # NCBI
        x = Classification(9606)
        x
        x.ids
        x.ncbi()
    """

    def __init__(self, ids):
        if isinstance(ids, int):
            ids = [ids]
        self.ids = ids

    def __repr__(self):
        x = """<%s>\n""" % type(self).__name__
        y = """  ids: %s""" % ",".join([str(w) for w in self.ids[:10]])
        return x + y

    def itis(self):
        out = []
        for i in range(len(self.ids)):
            id = self.ids[i]
            res = hierarchy_full(id)
            if res[0] is None:
                warnings.warn("No results for taxon '" + str(id) + "'")
                res = {}
            out.append(res)
        out = dict(zip(self.ids, out))
        return out

    def ncbi(self):
        res = hierarchy(self.ids)
        # out = []
        # for i in range(len(self.ids)):
        #     id = self.ids[i]
            # if res[0] is None:
            #     warnings.warn("No results for taxon '" + str(id) + "'")
            #     res = {}
            # out.append(res)
        # out = dict(zip(self.ids, out))
        return res

In [65]:
# Peut être utile pour obtenir les autres noms d'une espèce
x = Ids('Moustique')
print("itis : ", x.itis())
print("ids : ", x.ids)
scicomm.sci2comm(x)

itis :  None
ids :  {'Moustique': [{'id': '125930', 'name': 'Culicidae', 'rank': 'species', 'uri': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=125930'}, {'id': '126095', 'name': 'Wyeomyia smithii', 'rank': 'species', 'uri': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=126095'}, {'id': '1026919', 'name': 'Croton moustiquensis', 'rank': 'species', 'uri': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=1026919'}]}


{'Moustique': ['mosquitoes', 'maringouins', 'moustiques']}

In [7]:
# Enregistrement du nom de l'espèce
nom = Ids('Apis mellifera')
nom.itis()
print("ids :", nom.ids)
# Extraction de l'id de l'espèce
for item in nom.ids[nom.name[0]]:
    if item['name'] == nom.name[0]:
        id = item['id']
        break
id = int(id)
print("id :", id)
# Obtenir la classification de l'espèce
code = Classification(id)
lignee = code.itis()
print(f"La classification de {nom.name[0]} est :")
lignee[id]

ids : {'Apis mellifera': [{'id': '154396', 'name': 'Apis mellifera', 'rank': 'species', 'uri': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=154396'}, {'id': '1131130', 'name': 'Apis mellifera carnica', 'rank': 'species', 'uri': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=1131130'}, {'id': '1131131', 'name': 'Apis mellifera cecropia', 'rank': 'species', 'uri': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=1131131'}, {'id': '1131133', 'name': 'Apis mellifera scutellata', 'rank': 'species', 'uri': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=1131133'}, {'id': '1131134', 'name': 'Apis mellifera major', 'rank': 'species', 'uri': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=1131134'}, {'id': '1131136', 'name': 'Apis mellifera ligustica', 'rank': 'species', 'uri': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_

[{'author': None,
  'parentName': '',
  'parentTsn': '',
  'rankName': 'Kingdom',
  'taxonName': 'Animalia',
  'tsn': '202423'},
 {'author': None,
  'parentName': 'Animalia',
  'parentTsn': '202423',
  'rankName': 'Subkingdom',
  'taxonName': 'Bilateria',
  'tsn': '914154'},
 {'author': None,
  'parentName': 'Bilateria',
  'parentTsn': '914154',
  'rankName': 'Infrakingdom',
  'taxonName': 'Protostomia',
  'tsn': '914155'},
 {'author': None,
  'parentName': 'Protostomia',
  'parentTsn': '914155',
  'rankName': 'Superphylum',
  'taxonName': 'Ecdysozoa',
  'tsn': '914158'},
 {'author': None,
  'parentName': 'Ecdysozoa',
  'parentTsn': '914158',
  'rankName': 'Phylum',
  'taxonName': 'Arthropoda',
  'tsn': '82696'},
 {'author': None,
  'parentName': 'Arthropoda',
  'parentTsn': '82696',
  'rankName': 'Subphylum',
  'taxonName': 'Hexapoda',
  'tsn': '563886'},
 {'author': None,
  'parentName': 'Hexapoda',
  'parentTsn': '563886',
  'rankName': 'Class',
  'taxonName': 'Insecta',
  'tsn': '9

In [38]:
hierarchy_full(tsn = 154396, as_dataframe=True)

,author,parentName,parentTsn,rankName,taxonName,tsn
0,None,,,Kingdom,Animalia,202423
1,None,Animalia,202423,Subkingdom,Bilateria,914154
2,None,Bilateria,914154,Infrakingdom,Protostomia,914155
3,None,Protostomia,914155,Superphylum,Ecdysozoa,914158
4,None,Ecdysozoa,914158,Phylum,Arthropoda,82696
5,None,Arthropoda,82696,Subphylum,Hexapoda,563886
6,None,Hexapoda,563886,Class,Insecta,99208
7,None,Insecta,99208,Subclass,Pterygota,100500
8,None,Pterygota,100500,Infraclass,Neoptera,563890
9,None,Neoptera,563890,Superorder,Holometabola,914213


In [8]:
# Obtention de l'ordre, de la famille et du genre
hierarchie = lignee[id]
for item in hierarchie:
    if item['rankName'] == 'Order':
        order_taxon_name = item['taxonName']
    if item['rankName'] == 'Family':
        family_taxon_name = item['taxonName']
    if item['rankName'] == 'Genus':
        genus_taxon_name = item['taxonName']

print("Ordre :", order_taxon_name)
print("Famille :", family_taxon_name)
print("Genre :", genus_taxon_name)

Ordre : Hymenoptera
Famille : Apidae
Genre : Apis


# Pipeline complète

## **Apis mellifera**

In [45]:
# Enregistrement du nom de l'espèce
nom = Ids('Apis mellifera')
nom.itis()
# Extraction de l'id de l'espèce
for item in nom.ids[nom.name[0]]:
    if item['name'] == nom.name[0]:
        id = item['id']
        break
id = int(id)
# Obtenir la classification de l'espèce
code = Classification(id)
lignee = code.itis()
# Obtention de l'ordre, de la famille, du genre et de l'espèce
for item in lignee[id]:
    if item['rankName'] == 'Order':
        order_taxon_name = item['taxonName']
    if item['rankName'] == 'Family':
        family_taxon_name = item['taxonName']
    if item['rankName'] == 'Genus':
        genus_taxon_name = item['taxonName']
    if item['rankName'] == 'Species':
        species_taxon_name = item['taxonName']
# Création d'un tableau avec les informations
df = pd.DataFrame({'Ordre': [order_taxon_name], 
                   'Famille': [family_taxon_name], 
                   'Genre': [genus_taxon_name], 
                   'Espèce': [species_taxon_name]})
df

,Ordre,Famille,Genre,Espèce
0,Hymenoptera,Apidae,Apis,Apis mellifera


In [ ]:
# Enregistrement du nom de l'espèce
nom = Ids('Apis mellifera')
nom.itis()
# Extraction de l'id de l'espèce
for item in nom.ids[nom.name[0]]:
    if item['name'] == nom.name[0]:
        id = item['id']
        break
id = int(id)
# Obtenir la classification de l'espèce
code = Classification(id)
lignee = code.itis()
# Obtention de l'ordre, de la famille, du genre et de l'espèce
for item in lignee[id]:
    if item['rankName'] == 'Order':
        order_taxon_name = item['taxonName']
    if item['rankName'] == 'Family':
        family_taxon_name = item['taxonName']
    if item['rankName'] == 'Genus':
        genus_taxon_name = item['taxonName']
    if item['rankName'] == 'Species':
        species_taxon_name = item['taxonName']
# Création d'un tableau avec les informations
df1 = pd.DataFrame({'Ordre': [order_taxon_name], 
                   'Famille': [family_taxon_name], 
                   'Genre': [genus_taxon_name], 
                   'Espèce': [species_taxon_name]})
df1

## **Bombus**

In [10]:
# Enregistrement du nom de l'espèce
nom = Ids('Bombus')
nom.itis()
# Extraction de l'id de l'espèce
for item in nom.ids[nom.name[0]]:
    if item['name'] == nom.name[0]:
        id = item['id']
        break
id = int(id)
# Obtenir la classification de l'espèce
code = Classification(id)
lignee = code.itis()
# Obtention de l'ordre, de la famille, du genre et de l'espèce
for item in lignee[id]:
    if item['rankName'] == 'Order':
        order_taxon_name = item['taxonName']
    elif item['rankName'] == 'Family':
        family_taxon_name = item['taxonName']
    elif item['rankName'] == 'Genus':
        genus_taxon_name = item['taxonName']
    elif item['rankName'] == 'Species':
        species_taxon_name = item['taxonName']
# Création d'un tableau avec les informations
new_data = {'Ordre': order_taxon_name, 
            'Famille': family_taxon_name, 
            'Genre': genus_taxon_name, 
            'Espèce': species_taxon_name}
df = df.append(new_data, ignore_index=True)
df

C:\Users\corne\AppData\Local\Temp\ipykernel_3068\709395751.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_data, ignore_index=True)


,Ordre,Famille,Genre,Espèce
0,Hymenoptera,Apidae,Apis,Apis mellifera
1,Hymenoptera,Apidae,Bombus,Bombus zonatus


## **Tettigonioidea** (écrit Tettigonoidea dans le tableau)

In [11]:
# Enregistrement du nom de l'espèce
nom = Ids('Tettigonioidea')
nom.itis()
# Extraction de l'id de l'espèce
for item in nom.ids[nom.name[0]]:
    if item['name'] == nom.name[0]:
        id = item['id']
        break
id = int(id)
# Obtenir la classification de l'espèce
code = Classification(id)
lignee = code.itis()
print(f"La classification de {nom.name[0]} est :")
lignee[id]

La classification de Tettigonioidea est :


[{'author': None,
  'parentName': '',
  'parentTsn': '',
  'rankName': 'Kingdom',
  'taxonName': 'Animalia',
  'tsn': '202423'},
 {'author': None,
  'parentName': 'Animalia',
  'parentTsn': '202423',
  'rankName': 'Subkingdom',
  'taxonName': 'Bilateria',
  'tsn': '914154'},
 {'author': None,
  'parentName': 'Bilateria',
  'parentTsn': '914154',
  'rankName': 'Infrakingdom',
  'taxonName': 'Protostomia',
  'tsn': '914155'},
 {'author': None,
  'parentName': 'Protostomia',
  'parentTsn': '914155',
  'rankName': 'Superphylum',
  'taxonName': 'Ecdysozoa',
  'tsn': '914158'},
 {'author': None,
  'parentName': 'Ecdysozoa',
  'parentTsn': '914158',
  'rankName': 'Phylum',
  'taxonName': 'Arthropoda',
  'tsn': '82696'},
 {'author': None,
  'parentName': 'Arthropoda',
  'parentTsn': '82696',
  'rankName': 'Subphylum',
  'taxonName': 'Hexapoda',
  'tsn': '563886'},
 {'author': None,
  'parentName': 'Hexapoda',
  'parentTsn': '563886',
  'rankName': 'Class',
  'taxonName': 'Insecta',
  'tsn': '9

In [12]:
# Obtention de l'ordre, de la famille, du genre et de l'espèce
for item in lignee[id]:
    if item['rankName'] == 'Order':
        order_taxon_name = item['taxonName']
    elif item['rankName'] == 'Family':
        family_taxon_name = item['taxonName']
    elif item['rankName'] == 'Genus':
        genus_taxon_name = item['taxonName']
    elif item['rankName'] == 'Species':
        species_taxon_name = item['taxonName']
# Création d'un tableau avec les informations
new_data = {'Ordre': order_taxon_name, 
            'Famille': family_taxon_name, 
            'Genre': genus_taxon_name, 
            'Espèce': species_taxon_name}
df = df.append(new_data, ignore_index=True)
df

C:\Users\corne\AppData\Local\Temp\ipykernel_3068\3446007282.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_data, ignore_index=True)


,Ordre,Famille,Genre,Espèce
0,Hymenoptera,Apidae,Apis,Apis mellifera
1,Hymenoptera,Apidae,Bombus,Bombus zonatus
2,Orthoptera,Tettigoniidae,Bombus,Bombus zonatus


Deux problèmes :
- les fautes d'orthographes sont rédhibitoires
> Chercher dans la littérature s'il y a des moyens de chercher les noms similaires
- les catégories à extraires sont difficiles selon ce qui est disponible ;
    - si on a le nom complet il n'y a pas de problème : voir Apis mellifera
    - si on a le genre : les fonctions vont quand même chercher à nous donner le nom d'une espèce (donne la valeur +1)
    > Je dois essayer de corriger ça  
    > C'est corrigé !
    - si on a le nom d'une superfamille, la classification ne se poursut que jusqu'à +1 (Famille) donc on obtient dans le tableau le genre et l'espèce précédente (ancienne valeur enregistrée)
    > Créer une fonction pour donner les nouvelles valeurs  
    > Ca marche !

In [8]:
tableau = pd.DataFrame(columns=['Ordre', 'Super famille', 'Famille', 'Sous famille', 'Genre', 'Espèce'])

def get_classification(name):
    """
    """
    # Enregistrement du nom de l'espèce
    nom = Ids(name)
    nom.itis()
    # Extraction de l'id de l'espèce
    for item in nom.ids[nom.name[0]]:
        if item['name'] == nom.name[0]:
            id = int(item['id'])
            break
    # Obtenir la classification de l'espèce
    code = Classification(id)
    lignee = code.itis()
    # Obtention de l'ordre, de la famille, du genre et de l'espèce
    order_name, super_family_name, family_name, sub_family_name, genus_name, species_name = '', '', '', '', '', ''
    for item in lignee[id]:
        if item['parentName'] != nom.name[0]:
            if item['rankName'] == 'Order':
                order_name = item['taxonName']
            elif item['rankName'] == 'Superfamily':
                super_family_name = item['taxonName']
            elif item['rankName'] == 'Family':
                family_name = item['taxonName']
            elif item['rankName'] == 'Subfamily':
                sub_family_name = item['taxonName']
            elif item['rankName'] == 'Genus':
                genus_name = item['taxonName']
            elif item['rankName'] == 'Species':
                species_name = item['taxonName']
        else:
            break
    # Création d'un tableau avec les informations
    new_data = {'Ordre': order_name, 
                'Super famille': super_family_name,
                'Famille': family_name, 
                'Sous famille': sub_family_name,
                'Genre': genus_name, 
                'Espèce': species_name}
    return new_data

In [32]:
data = get_classification('Apis mellifera')
tableau = pd.concat([tableau, pd.DataFrame(data, index=[0])], ignore_index=True)
tableau

,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera


In [33]:
data = get_classification('Bombus')
tableau = pd.concat([tableau, pd.DataFrame(data, index=[0])], ignore_index=True)
tableau

,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
1,Hymenoptera,Apoidea,Apidae,Apinae,Bombus,


In [34]:
data = get_classification('Tettigonioidea')
tableau = pd.concat([tableau, pd.DataFrame(data, index=[0])], ignore_index=True)
tableau

,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
1,Hymenoptera,Apoidea,Apidae,Apinae,Bombus,
2,Orthoptera,Tettigonioidea,,,,


In [46]:
data = get_classification('Neomyia')
tableau = pd.concat([tableau, pd.DataFrame(data, index=[0])], ignore_index=True)
tableau

,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
1,Hymenoptera,Apoidea,Apidae,Apinae,Bombus,
2,Orthoptera,Tettigonioidea,,,,
3,Orthoptera,,,,,
4,Diptera,,Muscidae,Muscinae,Neomyia,


In [47]:
data = get_classification('Calliphora')
tableau = pd.concat([tableau, pd.DataFrame(data, index=[0])], ignore_index=True)
tableau

,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
1,Hymenoptera,Apoidea,Apidae,Apinae,Bombus,
2,Orthoptera,Tettigonioidea,,,,
3,Orthoptera,,,,,
4,Diptera,,Muscidae,Muscinae,Neomyia,
5,Diptera,,Calliphoridae,Calliphorinae,Calliphora,


In [7]:
def get_hierarchie(name):
    """
    """
    # Enregistrement du nom de l'espèce
    nom = Ids(name)
    nom.itis()
    # Extraction de l'id de l'espèce
    for item in nom.ids[nom.name[0]]:
        if item['name'] == nom.name[0]:
            id = item['id']
            break
    # Obtenir la classification de l'espèce
    return hierarchy_full(id, as_dataframe=True)

In [38]:
df_neom = get_hierarchie('Cucujidae')

In [39]:
df_call = get_hierarchie('Cryptophagidae')

In [40]:
intersection_df = df_neom.merge(df_call, how='inner')
intersection_df

,author,parentName,parentTsn,rankName,taxonName,tsn
0,None,,,Kingdom,Animalia,202423
1,None,Animalia,202423,Subkingdom,Bilateria,914154
2,None,Bilateria,914154,Infrakingdom,Protostomia,914155
3,None,Protostomia,914155,Superphylum,Ecdysozoa,914158
4,None,Ecdysozoa,914158,Phylum,Arthropoda,82696
5,None,Arthropoda,82696,Subphylum,Hexapoda,563886
6,None,Hexapoda,563886,Class,Insecta,99208
7,None,Insecta,99208,Subclass,Pterygota,100500
8,None,Pterygota,100500,Infraclass,Neoptera,563890
9,None,Neoptera,563890,Superorder,Holometabola,914213


### Remplacer :
- Les Mouches difficiles à déterminer par **Diptera**
- Les Coléoptères difficiles à déterminer par **Coleoptera**
- Les Moustiques, Tipules et autres diptères Nématocères par **Nematocera**
- Les Chenilles et fausses-Chenilles par **Lepidoptera**
- Les Syrphes difficiles à déterminer par **Syrphinae**
- Les Terebrants Chalcidiens et autres par **Chalcidoidea**
- Les Epeires et autres Araneidae par **Araneidae**
- Les Punaises difficiles à déterminer par **Heteroptera**
- Les Syrphes aux fémurs enflés par **Syrphinae**
- Les Tachinaires difficiles à déterminer par **Tachinidae**
- Les Punaises prédatrices ternes par **Lygus lineolaris** (Punaise terne)